In [ ]:
def prepare_data(sequence, sequence_d1, sequence_d2, labels, options, maxlen=None, max_word=100):
    # length = [len(s) for s in sequence]
    length, length_d1, length_d2 = [], [], [] #一天里面 number of news
    for i, d1, d2 in zip(sequence, sequence_d1, sequence_d2): #x,one batch里面one day
        dd1, dd2 = list(), list()
        length.append(len(i))
        for day in d1:
            dd1.append(len(day))
        length_d1.append(dd1)
        for day in d2:
            dd2.append(len(day))
        length_d2.append(dd2)
    if maxlen is not None:  # max length is the news level
        new_sequence = []
        new_lengths = []
        new_sequence_d1 = []
        new_lengths_d1 = []
        new_sequence_d2 = []
        new_lengths_d2 = []
        for l, s, ld1, sd1, ld2, sd2 in zip(length, sequence, length_d1, sequence_d1, length_d2, sequence_d2):
            dd1, lld1, dd2, lld2 = list(), list(), list(), list()
            if l < maxlen:
                new_sequence.append(s)
                new_lengths.append(l)
            for i, j in zip(ld1, sd1):
                if i < maxlen:
                    dd1.append(j)
                    lld1.append(i)
            new_sequence_d1.append(dd1)
            new_lengths_d1.append(lld1)
            for i, j in zip(ld2, sd2):
                if i < maxlen:
                    dd2.append(j)
                    lld2.append(i)
            new_sequence_d2.append(dd2)
            new_lengths_d2.append(lld2)

        length = new_lengths  # This step is to filter the sentence which length is bigger
        sequence = new_sequence  # than the max length. length means number of news. sequence means 
        # length of each sentence
        length_d1 = new_lengths_d1
        sequence_d1 = new_sequence_d1
        length_d2 = new_lengths_d2
        sequence_d2 = new_sequence_d2
        ##TODO need to be careful, set the max length bigger to avoid bug
        if len(length) < 1:
            return None, None, None, None, None, None, None, None
    
    day1 = options['delay1']-1
    day2 = options['delay2']-options['delay1']
    maxlen_x = numpy.max(length)  # max time step
    try:
        maxlen_xd1 = numpy.max([numpy.max(i) for i in length_d1])
        maxlen_xd2 = numpy.max([numpy.max(i) for i in length_d2])
    except ValueError as e:
        print(str(e))
        maxlen_xd1=100
        maxlen_xd2=100
    n_samples = len(sequence)  # number of samples== batch
    max_sequence = max(len(j) for i in sequence for j in i)  # find the sequence max length
    max_sequence_d1 = max(len(j) for i in sequence_d1 for z in i for j in z)
    max_sequence_d2 = max(len(j) for i in sequence_d2 for z in i for j in z)
    max_sequence = max_word if max_sequence > max_word else max_sequence  # shrink the data size
    max_sequence_d1 = max_word if max_sequence_d1 > max_word else max_sequence_d1  # shrink the data size
    max_sequence_d2 = max_word if max_sequence_d2 > max_word else max_sequence_d2  # shrink the data size
    ##TODO for x
    x = numpy.zeros((n_samples, maxlen_x, max_sequence)).astype('int64')
    x_mask = numpy.zeros((n_samples, maxlen_x)).astype('float32')
    ##TODO for x_d1
    x_d1 = numpy.zeros((n_samples, day1, maxlen_xd1, max_sequence_d1)).astype('int64')
    x_d1_mask = numpy.zeros((n_samples, day1, maxlen_xd1)).astype('float32')
    ##TODO for x_d2
    x_d2 = numpy.zeros((n_samples, day2, maxlen_xd2, max_sequence_d2)).astype('int64')
    x_d2_mask = numpy.zeros((n_samples, day2, maxlen_xd2)).astype('float32')
    final_mask = numpy.ones((n_samples, 1 + day1 + day2)).astype('float32')
    # l = numpy.array(labels).astype('int64')
    ##TODO for label
    l = numpy.zeros((n_samples,)).astype('int64')
    for index, (i, j, k, ll) in enumerate(zip(sequence, sequence_d1, sequence_d2, labels)):  # batch size
        l[index] = ll
        for idx, ss in enumerate(i):  # time step
            if len(ss) < max_sequence:
                x[index, idx, :len(ss)] = ss
            else:
                x[index, idx, :max_sequence] = ss[:max_sequence]
            x_mask[index, idx] = 1.
        for jj, day in enumerate(j):
            for idx, ss in enumerate(day):
                if len(ss) < max_sequence_d1:
                    x_d1[index, jj, idx, :len(ss)] = ss
                else:
                    x_d1[index, jj, idx, :max_sequence_d1] = ss[:max_sequence_d1]
                x_d1_mask[index, jj, idx] = 1.
        for kk, day in enumerate(k):
            for idx, ss in enumerate(day):
                if len(ss) < max_sequence_d2:
                    x_d2[index, kk, idx, :len(ss)] = ss
                else:
                    x_d2[index, kk, idx, :max_sequence_d2] = ss[:max_sequence_d2]
                x_d2_mask[index, kk, idx] = 1.
    
    #################3 maxlen_x 都小于 maxlen(100),但每一组maxlen_x都不一样！！
    return x, x_mask, x_d1, x_d1_mask, x_d2, x_d2_mask, l, final_mask

In [ ]:
#有超参数temp,dim=150/500-->options['attn_pooling'];inst_pred的activation func
def milrepre(emb, sequence_mask, instance_mask, keep_prob, is_training, options):
    # emb batch,news, sequence,embedding, 32*40*13*100
    # sequence_mask batch, news,sequence 32*40*13
    # instance_mask batch, news, 32*40
    batch = tf.shape(emb)[0]
    N = tf.shape(emb)[1]
    word = tf.shape(emb)[2]
    word_level_inputs = tf.reshape(emb, [batch * N, word, options['dim_word']])
    word_level_mask = tf.reshape(sequence_mask, [batch * N, word])
    ##TODO average word
    word_level_output = tf.reduce_sum(word_level_inputs * tf.expand_dims(word_level_mask, -1), 1) / tf.expand_dims(
        tf.reduce_sum(word_level_mask, 1) + 1e-8, 1)
    # word_level_output shape is (32*40)*100
    att = tf.reshape(word_level_output, [batch, N, options['dim']])
 
    ##TODO MIL bag_representation
    instance_temp = tf.layers.dense(word_level_output, options['attn_pooling'], activation=tf.nn.tanh,use_bias=True,kernel_initializer=layers.xavier_initializer(uniform=True,seed=None,dtype=tf.float32),name='inst_temp', reuse=tf.AUTO_REUSE)
    if options['use_dropout']:
        instance_temp = layers.dropout(instance_temp, keep_prob=keep_prob, is_training=is_training,seed=None)
    pred_ = tf.layers.dense(instance_temp, 1, activation=None, use_bias=True,kernel_initializer=layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32),name='inst_pred', reuse=tf.AUTO_REUSE)
    instance_pred = tf.nn.softmax(pred_)#32*N,1 NOT 32,N,1, float32   
    instance_level_input = tf.reshape(instance_pred,[batch,N,1])           
    instance_level_input = instance_level_input * tf.expand_dims(instance_mask, -1)  # mask before attention
    ##TODO bag_representation
    coef = tf.concat([instance_level_input for i in range(options['dim'])], 2)
    bag_repre = tf.multiply(coef, att)
    bag_repre = tf.reduce_mean(bag_repre, axis=1) #32,100
    
    return bag_repre

In [ ]:
def milrepre_day(emb, sequence_mask, instance_mask, keep_prob, is_training, options):
    # emb batch,day,news, sequence,embedding, 32*3*40*13*100
    # sequence_mask batch, day, news,sequence 32*3*40*13
    # instance_mask batch, day, news, 32*3*40
    batch = tf.shape(emb)[0]
    day = tf.shape(emb)[1]
    N = tf.shape(emb)[2]
    word = tf.shape(emb)[3]
    word_level_inputs = tf.reshape(emb, [batch * day * N, word, options['dim_word']])
    word_level_mask = tf.reshape(sequence_mask, [batch * day * N, word])
    instances_level_mask = tf.reshape(instance_mask, [batch * day, N])
    """##TODO word level LSTM
    word_encoder_out = bilstm_filter(word_level_inputs, word_level_mask, keep_prob,
                                     prefix='sequence_encode', dim=options['dim'],
                                     is_training=is_training)  # output shape: batch*day*news,sequence,2*lstm_units(32*3*40)*12*600
    word_encoder_out = tf.concat(word_encoder_out, 2) * tf.expand_dims(word_level_mask, -1)  # mask the output
    ##TODO word level attention
    word_level_output = attention_v2(word_encoder_out, word_level_mask, name='word_attention', keep=keep_prob, r=10,
                                     is_training=is_training)
    # word_level_output shape is (32*3*40)*600
    if options['use_dropout']:
        word_level_output = layers.dropout(word_level_output, keep_prob=keep_prob, is_training=is_training,seed=None)
    """
    ##TODO average word
    word_level_output = tf.reduce_sum(word_level_inputs * tf.expand_dims(word_level_mask, -1), 1) / tf.expand_dims(
        tf.reduce_sum(word_level_mask, 1) + 1e-8, 1)
    # word_level_output shape is (32*3*40)*100
    att = tf.reshape(word_level_output, [batch * day, N, options['dim']])  # (32*3),40,100  
    
    ##TODO MIL bag_representation
    instance_temp = tf.layers.dense(word_level_output, options['attn_pooling'], activation=tf.nn.tanh,use_bias=True,kernel_initializer=layers.xavier_initializer(uniform=True,seed=None,dtype=tf.float32),name='inst_temp', reuse=tf.AUTO_REUSE)
    if options['use_dropout']:
        instance_temp = layers.dropout(instance_temp, keep_prob=keep_prob, is_training=is_training,seed=None)
    pred_ = tf.layers.dense(instance_temp, 1, activation=None, use_bias=True,kernel_initializer=layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32),name='inst_pred', reuse=tf.AUTO_REUSE)
    instance_pred = tf.nn.softmax(pred_)  
    instance_level_input = tf.reshape(instance_pred,[batch*day,N,1])   #32*day,N,1, float32    
    instance_level_input = instance_level_input * tf.expand_dims(instances_level_mask, -1)  # mask before attention
    ##TODO bag_representation
    coef = tf.concat([instance_level_input for i in range(options['dim'])], 2)
    bag_repre = tf.multiply(coef, att)
    bag_repre = tf.reduce_mean(bag_repre, axis=1) #32*day,100
    day_level_output = tf.reshape(bag_repre, [batch, day, options['dim']])  # (32*3)*100

    return day_level_output #32,3,100


##############Questions : instance_level_input = instance_level_input * tf.expand_dims(instances_level_mask, -1)??????????需不需要？？

In [ ]:
def build_model(embedding, options):
    """ Builds the entire computational graph used for training
    """
    # description string: #words x #samples
    with tf.device('/gpu:1'):
        with tf.variable_scope('input'):
            x = tf.placeholder(tf.int64, shape=[None, None, None],
                               name='x')  # 3D vector batch,news and sequence(before embedding)40*32*13
            x_mask = tf.placeholder(tf.float32, shape=[None, None], name='x_mask')  # mask batch,news
            y = tf.placeholder(tf.int64, shape=[None], name='y')
            x_d1 = tf.placeholder(tf.int64, shape=[None, None, None, None], name='x_d1')
            x_d1_mask = tf.placeholder(tf.float32, shape=[None, None, None], name='x_d1_mask')
            x_d2 = tf.placeholder(tf.int64, shape=[None, None, None, None], name='x_d2')
            x_d2_mask = tf.placeholder(tf.float32, shape=[None, None, None], name='x_d2_mask')
            final_mask = tf.placeholder(tf.float32, shape=[None, None], name='final_mask')
            tech = tf.placeholder(tf.float32, shape=[None, None,7], name='technical') #shape is batch time unit
            # final_mask shape is day*n_samples
            ##TODO important    
            keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
            is_training = tf.placeholder(tf.bool, name='is_training')
            ##TODO important
            sequence_mask = tf.cast(tf.abs(tf.sign(x)), tf.float32)  # 3D
            sequence_d1_mask = tf.cast(tf.abs(tf.sign(x_d1)), tf.float32)  # 4D
            sequence_d2_mask = tf.cast(tf.abs(tf.sign(x_d2)), tf.float32)  # 4D
            n_timesteps = tf.shape(x)[0]  # time steps
            #n_samples = tf.shape(x)[1]  # n samples
            # # word embedding
            ##TODO word embedding
            emb = tf.nn.embedding_lookup(embedding, x)
            emb_d1 = tf.nn.embedding_lookup(embedding, x_d1)
            emb_d2 = tf.nn.embedding_lookup(embedding, x_d2)
            
    with tf.device('/gpu:1'):
        # fed into the input of BILSTM from the official document
        ##TODO word level LSTM
        with tf.name_scope('news'):
            att = news(emb, sequence_mask, x_mask, keep_prob, is_training, options)
        ##TODO att shape 32*600 att_day1 32*3*600 att_day2 32*4*600
        with tf.name_scope('day1'):
            att_day1 = days(emb_d1, sequence_d1_mask, x_d1_mask, keep_prob, is_training, options)
        # TODO bilstm layers
        # Change the time step and batch
    with tf.device('/gpu:1'):
        with tf.name_scope('day2'):
            att_day2 = days(emb_d2, sequence_d2_mask, x_d2_mask, keep_prob, is_training, options)
        with tf.name_scope('final'):
            final = tf.concat([att_day2, att_day1, tf.expand_dims(att, 1)], 1)
            '''if options['use_dropout']:
                final = layers.dropout(final, keep_prob=keep_prob, is_training=is_training)
            '''
            # final shape is 8*32*600
            if options['last_layer'] == 'LSTM':
                final = bilstm_filter(final, final_mask, keep_prob, prefix='day_lstm', dim=100,
                                    is_training=is_training)  # output shape: batch,time_step,2*lstm_unit(concate) 32*7*600
                #tech_ind = lstm_filter(tech, tf.ones(shape=[tf.shape(tech)[0],tf.shape(tech)[1]]), keep_prob, prefix='tech_lstm', dim=50,
                #                    is_training=is_training)
                ##TODO day level attention
                att_final = attention_v2(tf.concat(final, 2), final_mask, name='day_attention', keep=keep_prob,r=4,
                                is_training=is_training)  # already masked after attention
                ##TODO take day lstm average
                # att_final = tf.reduce_mean(tf.concat(final,2),1)
                # tech_att = tf.reduce_mean(tf.concat(tech_ind,2),1)
                ##TODO take the lasts
                #tech_att=tech_ind[:,-1,:]
                #att_final = tf.concat([att_final,tech_att],axis=1)
                logit = tf.layers.dense(att_final, 100, activation=tf.nn.tanh, use_bias=True,
                                        kernel_initializer=layers.xavier_initializer(uniform=True, seed=None,
                                                                                    dtype=tf.float32),
                                        name='ff', reuse=tf.AUTO_REUSE)
                # logit = tf.layers.batch_normalization(logit, training=is_training)
                # logit=tf.nn.tanh(logit)

                '''
                # logit1 = tf.reduce_sum(tf.concat(final,2) * tf.expand_dims(final_mask,-1),0) / tf.expand_dims(tf.reduce_sum(final_mask,0),1)
                # logit2 = tf.reduce_max(ctx3 * tf.expand_dims(x1_mask,2),0)
                '''
            if options['last_layer'] == 'CNN':
                att_ctx = tf.concat([att_day1, tf.expand_dims(att, 1)], 1)
                xavier = layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32)
                conv1 = tf.layers.conv1d(att_ctx, filters=options['CNN_filter'],
                                        kernel_size=options['CNN_kernel'], padding='same', strides=1,
                                        activation=tf.nn.relu, kernel_initializer=xavier, name='conv1')
                conv2 = tf.layers.conv1d(final, filters=options['CNN_filter'],
                                        kernel_size=options['CNN_kernel'], padding='same',
                                        strides=1, activation=tf.nn.relu,
                                        kernel_initializer=xavier,
                                        name='conv2')

                pool1 = tf.layers.max_pooling1d(conv1, pool_size=2, strides=2, padding='same',
                                                data_format='channels_last', name='pool1')
                pool2 = tf.layers.max_pooling1d(conv2, pool_size=2, strides=2, padding='same',
                                                data_format='channels_last', name='pool2')
                d1size = math.ceil(options['delay1'] / 2) * options['CNN_filter']
                d2size = math.ceil(options['delay2'] / 2) * options['CNN_filter']
                pool1_flat = tf.reshape(pool1, [-1, d1size])
                pool2_flat = tf.reshape(pool2, [-1, d2size])
                cnn_final = tf.concat([att, pool1_flat, pool2_flat], -1)
                logit = tf.layers.dense(cnn_final, 300, activation=tf.nn.tanh, use_bias=True,
                                        kernel_initializer=layers.xavier_initializer(uniform=True, seed=None,
                                                                                    dtype=tf.float32),
                                        name='ff', reuse=tf.AUTO_REUSE)
                # logit = tf.layers.batch_normalization(logit, training=is_training)
                # logit=tf.nn.tanh(logit)

            if options['use_dropout']:
                logit = layers.dropout(logit, keep_prob=keep_prob, is_training=is_training,seed=None)
            pred = tf.layers.dense(logit, 2, activation=None, use_bias=True,
                                kernel_initializer=layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32),
                                name='fout', reuse=tf.AUTO_REUSE)
            logger.info('Building f_cost...')
            # todo not same
            labels = tf.one_hot(y, depth=2, axis=1)
            # labels = y
            preds = tf.nn.softmax(pred, 1,name='softmax')
            # preds = tf.nn.sigmoid(pred)
            # pred=tf.reshape(pred,[-1])
            cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=labels)
            # cost = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels,logits=pred),1)
            # cost = -tf.reduce_sum((tf.cast(labels, tf.float32) * tf.log(preds + 1e-8)),axis=1)
            #cost = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=y)
        logger.info('Done')
        '''
        logit1 = tf.reduce_sum(ctx1 * tf.expand_dims(x_mask, 2), 0) / tf.expand_dims(tf.reduce_sum(x_mask, 0), 1)
        logit2 = tf.reduce_max(ctx1 * tf.expand_dims(x_mask, 2), 0)
        logit = tf.concat([logit1, logit2], 1)
        '''

        with tf.variable_scope('logging'):
            tf.summary.scalar('current_cost', tf.reduce_mean(cost))
            tf.summary.histogram('predicted_value', preds)
            summary = tf.summary.merge_all()

    return is_training, cost, x, x_mask, y, n_timesteps, preds, summary